## Import dataset

In [1]:
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

tf.logging.set_verbosity(tf.logging.ERROR)  # suppress some deprecation warnings

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')

/cluster/apps/python/3.6.4/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
import os

data_directory = './data'

# data_train = pd.read_csv(os.path.join(data_directory, 'train_stories.csv'), header='infer')

data_val = pd.read_csv(os.path.join(data_directory, 'cloze_test_val__spring2016 - cloze_test_ALL_val.csv'), header='infer')
data_test = pd.read_csv(os.path.join(data_directory, 'cloze_test_test__spring2016 - cloze_test_ALL_test.csv'), header='infer')

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np

W0523 21:10:08.796071 47570318003648 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [5]:
OUTPUT_DIR = 'output_dir'
DO_DELETE = False
USE_BUCKET = False

In [6]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /cluster/home/sanagnos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /cluster/home/sanagnos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
def create_dataset(data_val):
    new_dataset = []
    classes = []
    for pos in range(len(data_val)):
        story_start = data_val.iloc[pos][['InputSentence' + str(i) for i in [1, 2, 3, 4]]].values
        
        new_dataset.append(" ".join(np.append(story_start, " ||| " + data_val.iloc[pos]['RandomFifthSentenceQuiz1'])))
        new_dataset.append(" ".join(np.append(story_start, " ||| " + data_val.iloc[pos]['RandomFifthSentenceQuiz2'])))
        
        if data_val.iloc[pos]['AnswerRightEnding'] == 1:
            classes.append(0)
            classes.append(1)
        else:
            classes.append(1)
            classes.append(0)
            
    return new_dataset, classes

dataset_val, classes_val = create_dataset(data_val)
val_pd = pd.DataFrame({'story': dataset_val, 'class': classes_val})

dataset_test, classes_test = create_dataset(data_test)
test_pd = pd.DataFrame({'story': dataset_test, 'class': classes_test})

In [8]:
from sklearn.utils import shuffle

# train_size = 0.80

# np.random.seed(42)
# msk = np.random.rand(len(test_pd) // 2) < train_size
# new_mask = np.array([[value, value] for value in msk]).reshape(-1)

# train = shuffle(test_pd[new_mask])

# test = test_pd[~new_mask]

train = shuffle(val_pd)
train_unshuffled = val_pd
test = test_pd

print(len(train))
print(len(test))

3742
3742


In [9]:
DATA_COLUMN = 'story'
LABEL_COLUMN = 'class'
# label_list is 0 for a true story and 1 for a false story
label_list = [0, 1]

# Use the InputExample class from BERT's run_classifier code to create examples from the data

train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [10]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [11]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 96
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [66]:
tf.reset_default_graph()

In [67]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value
  n_ctx = input_ids.shape[-1].value
  
#   THIS WORKS PRETTY WELL alsmost 90 ensemble with small bert
#   transformer_outputs = bert_outputs['sequence_output']
#   output_layer = transformer_outputs[:, -8:]
#   print(tf.shape(bert_outputs["pooled_output"][-1]))
#   print(output_layer)
#   output_layer = tf.concat(output_layer, axis=1)
#   print(output_layer)    
#   output_layer = tf.reshape(output_layer, [-1, 768 * 8])
#   print(output_layer)   

    
  transformer_outputs = bert_outputs['sequence_output']

  weights_size = hidden_size * 5
  # These weights are used to multiply all transformer outputs after the separator
  output_weights = tf.get_variable("output_weights", [weights_size, hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable("output_bias", [weights_size], initializer=tf.zeros_initializer())
  
  clf_token = 1064

  tf_argmax = tf.argmax(tf.cast(tf.equal(input_ids, clf_token), tf.float32), 1)

    
  def create_new_state(x):
    # remove the 3 separators ... or not ?
    i = x[0]
    hidden_pos = x[1]
    result = tf.zeros([weights_size])
    
    condition = lambda i, m: i < n_ctx
#     body = lambda i, m: [i+1, m + output_weights * hidden_pos[i]]
    # 
    body = lambda i, m: [i+1, m + tf.reshape(tf.matmul(output_weights, tf.expand_dims(hidden_pos[i], 1)), [-1])]



    return tf.while_loop(condition, body, loop_vars=[i, result],
                         shape_invariants=[i.get_shape(), tf.TensorShape([weights_size])])[1]

  output_layer = tf.map_fn(create_new_state, (tf_argmax, transformer_outputs), dtype=tf.float32) + output_bias

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
    logits = tf.layers.dense(output_layer, 512, use_bias=True, activation=tf.nn.sigmoid)
    logits = tf.nn.dropout(logits, keep_prob=0.9)
    logits = tf.layers.dense(logits, num_labels, use_bias=True)

    log_probs = tf.nn.log_softmax(logits, axis=-1)
    
    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [68]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        return {
            "eval_accuracy": accuracy,
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [69]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 15.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 50000
SAVE_SUMMARY_STEPS = 100000

OUTPUT_DIR = 'output_dir'
SAVE_RESULTS_DIR = 'results_predictions'

N_ESTIMATORS = 15

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print('num_train_steps', num_train_steps)

# Skip this step to avoid disk quota
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

num_train_steps 3508


In [70]:
def get_final_predictions(in_sentences):
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    predictions = [prediction['probabilities'] for prediction in predictions]

    return predictions

def combine_predictions(predictions):
    my_predictions = []

    i = 0
    while i < len(test):
        p_first = np.exp(predictions[i])
        p_second = np.exp(predictions[i + 1])

        p1 = p_first[0] + p_second[1]
        p2 = p_first[1] + p_second[0]

        if p1 > p2:
            my_predictions.append(1)
        else:
            my_predictions.append(2)
        i += 2
        
    return np.array(my_predictions)

In [ ]:
!rm -rf {SAVE_RESULTS_DIR} || true
!mkdir {SAVE_RESULTS_DIR}

true_labels_train = train_unshuffled['class'].values[::2] + 1
true_labels_val = test['class'].values[::2] + 1

for i in range(N_ESTIMATORS):
    !rm -rf {OUTPUT_DIR} || true
    
    train_features = shuffle(train_features)
    
    # Create an input function for training. drop_remainder = True for using TPUs.
    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    
    print(f'========= BEGGINING TRAINING FOR CLASSIFIER {i:2d} =========')
    current_time = datetime.now()
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    print("Training took time ", datetime.now() - current_time)
        
    predictions = get_final_predictions(train_unshuffled['story'].values)
    print('Score on train is ', accuracy_score(true_labels_train, combine_predictions(predictions)))
    np.savetxt(os.path.join("./" + SAVE_RESULTS_DIR, "predictions_train_" + str(i) + '.csv'), predictions, delimiter=",")
    
    predictions = get_final_predictions(test['story'].values)
    val_score = accuracy_score(true_labels_val, combine_predictions(predictions))
    print('Score on val is ', val_score)
    np.savetxt(os.path.join("./" + SAVE_RESULTS_DIR, "predictions_test_" + str(val_score) + '_classifier_' + str(i) + '.csv'), predictions, delimiter=",")

========= BEGGINING TRAINING FOR CLASSIFIER  0 =========


/cluster/home/sanagnos/.local/lib64/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training took time  0:39:06.473375
Score on train is  1.0


In [10]:
from os import listdir
from os.path import isfile, join
from scipy import stats
import numpy as np

SAVE_RESULTS_DIR = 'results_predictions'


files = [f for f in listdir(SAVE_RESULTS_DIR) if isfile(join(SAVE_RESULTS_DIR, f))]

true_labels_train = train_unshuffled['class'].values[::2] + 1
true_labels_test = test['class'].values[::2] + 1

classifiers = [int(file.split("_")[2].split(".")[0]) for file in files]
num_classifiers = np.max(classifiers)

predictions_train = []
predictions_test = []
for i in range(num_classifiers + 1):
    predictions_file_train = np.genfromtxt(os.path.join("./" + SAVE_RESULTS_DIR, 
                                                        "predictions_train_" + str(i) + '.csv'), delimiter=',')
    predictions_train.append(predictions_file_train)
    
    predictions_file_test = np.genfromtxt(os.path.join("./" + SAVE_RESULTS_DIR, 
                                                       "predictions_test_" + str(i) + '.csv'), delimiter=',')
    predictions_test.append(predictions_file_test)
    print(f'For classifier {i:2d} train accuracy {accuracy_score(true_labels_train, combine_predictions(predictions_file_train)):.4f} and test accuracy {accuracy_score(true_labels_test, combine_predictions(predictions_file_test)):.4f}')
    
def print_ensemble_predictions(predictions, true_labels):
    preds_mode = [combine_predictions(p) for p in predictions]
    preds_mode = np.array(preds_mode)
    preds_mode = stats.mode(preds_mode)[0][0]

    print('ENSEMBLE ACCURACY MODE')
    print(accuracy_score(true_labels, preds_mode))

    preds_prob = np.mean(predictions, axis=0)
    preds_prob = combine_predictions(preds_prob)

    print('ENSEMBLE ACCURACY PROB MEAN ON LOGS')
    print(accuracy_score(true_labels, preds_prob))


    preds_prob = np.log(np.mean(np.exp(predictions), axis=0))
    preds_prob = combine_predictions(preds_prob)

    print('ENSEMBLE ACCURACY PROB MEAN ON PROBS')
    print(accuracy_score(true_labels, preds_prob))
    print()
    
    
print_ensemble_predictions(predictions_train, true_labels_train)
print_ensemble_predictions(predictions_test, true_labels_test)

For classifier  0 train accuracy 1.0000 and test accuracy 0.8792
For classifier  1 train accuracy 1.0000 and test accuracy 0.8787
For classifier  2 train accuracy 0.9989 and test accuracy 0.8952
For classifier  3 train accuracy 1.0000 and test accuracy 0.8803
For classifier  4 train accuracy 1.0000 and test accuracy 0.8632
For classifier  5 train accuracy 1.0000 and test accuracy 0.8717
For classifier  6 train accuracy 1.0000 and test accuracy 0.8797
For classifier  7 train accuracy 1.0000 and test accuracy 0.8733
For classifier  8 train accuracy 1.0000 and test accuracy 0.8781
For classifier  9 train accuracy 1.0000 and test accuracy 0.8712
For classifier 10 train accuracy 1.0000 and test accuracy 0.8616
For classifier 11 train accuracy 1.0000 and test accuracy 0.8669
For classifier 12 train accuracy 0.9995 and test accuracy 0.8589
For classifier 13 train accuracy 1.0000 and test accuracy 0.8632
For classifier 14 train accuracy 1.0000 and test accuracy 0.8803
ENSEMBLE ACCURACY MODE
1.

In [67]:
small_bert (whole val and whole test)

output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
logits = tf.layers.dense(output_layer, 128, use_bias=True, activation=tf.nn.sigmoid)
logits = tf.nn.dropout(logits, keep_prob=0.9)
logits = tf.layers.dense(logits, num_labels, use_bias=True)

For classifier  0 train accuracy 1.0000 and test accuracy 0.8391
For classifier  1 train accuracy 1.0000 and test accuracy 0.8343
For classifier  2 train accuracy 1.0000 and test accuracy 0.8386
For classifier  3 train accuracy 1.0000 and test accuracy 0.8311
For classifier  4 train accuracy 1.0000 and test accuracy 0.8434
For classifier  5 train accuracy 1.0000 and test accuracy 0.8397
For classifier  6 train accuracy 1.0000 and test accuracy 0.8434
For classifier  7 train accuracy 1.0000 and test accuracy 0.8418
For classifier  8 train accuracy 1.0000 and test accuracy 0.8359
For classifier  9 train accuracy 1.0000 and test accuracy 0.8487
For classifier 10 train accuracy 1.0000 and test accuracy 0.8332
For classifier 11 train accuracy 1.0000 and test accuracy 0.8429
For classifier 12 train accuracy 1.0000 and test accuracy 0.8397
For classifier 13 train accuracy 1.0000 and test accuracy 0.8407
For classifier 14 train accuracy 1.0000 and test accuracy 0.8365
ENSEMBLE ACCURACY MODE
1.0
ENSEMBLE ACCURACY PROB MEAN ON LOGS
1.0
ENSEMBLE ACCURACY PROB MEAN ON PROBS
1.0

ENSEMBLE ACCURACY MODE
0.8722608230892571
ENSEMBLE ACCURACY PROB MEAN ON LOGS
0.8786745056119722
ENSEMBLE ACCURACY PROB MEAN ON PROBS
0.8786745056119722

big_bert (whole val and whole test)
For classifier  0 train accuracy 1.0000 and test accuracy 0.8691
For classifier  1 train accuracy 1.0000 and test accuracy 0.8664
For classifier  2 train accuracy 1.0000 and test accuracy 0.8680
For classifier  3 train accuracy 1.0000 and test accuracy 0.8562
For classifier  4 train accuracy 0.9984 and test accuracy 0.8397
For classifier  5 train accuracy 1.0000 and test accuracy 0.8696
For classifier  6 train accuracy 1.0000 and test accuracy 0.8381
For classifier  7 train accuracy 1.0000 and test accuracy 0.8589
For classifier  8 train accuracy 1.0000 and test accuracy 0.8632
For classifier  9 train accuracy 1.0000 and test accuracy 0.8600
For classifier 10 train accuracy 1.0000 and test accuracy 0.8701
For classifier 11 train accuracy 1.0000 and test accuracy 0.8696
For classifier 12 train accuracy 0.9947 and test accuracy 0.8268
For classifier 13 train accuracy 1.0000 and test accuracy 0.8653
For classifier 14 train accuracy 0.9995 and test accuracy 0.8044
ENSEMBLE ACCURACY MODE
1.0
ENSEMBLE ACCURACY PROB MEAN ON LOGS
1.0
ENSEMBLE ACCURACY PROB MEAN ON PROBS
1.0

ENSEMBLE ACCURACY MODE
0.8979155531801176
ENSEMBLE ACCURACY PROB MEAN ON LOGS
0.9021913415285944
ENSEMBLE ACCURACY PROB MEAN ON PROBS
0.8984500267236771

big bert [512]

For classifier  0 train accuracy 1.0000 and test accuracy 0.8792
For classifier  1 train accuracy 1.0000 and test accuracy 0.8787
For classifier  2 train accuracy 0.9989 and test accuracy 0.8952
For classifier  3 train accuracy 1.0000 and test accuracy 0.8803
For classifier  4 train accuracy 1.0000 and test accuracy 0.8632
For classifier  5 train accuracy 1.0000 and test accuracy 0.8717
For classifier  6 train accuracy 1.0000 and test accuracy 0.8797
For classifier  7 train accuracy 1.0000 and test accuracy 0.8733
For classifier  8 train accuracy 1.0000 and test accuracy 0.8781
For classifier  9 train accuracy 1.0000 and test accuracy 0.8712
For classifier 10 train accuracy 1.0000 and test accuracy 0.8616
For classifier 11 train accuracy 1.0000 and test accuracy 0.8669
For classifier 12 train accuracy 0.9995 and test accuracy 0.8589
For classifier 13 train accuracy 1.0000 and test accuracy 0.8632
For classifier 14 train accuracy 1.0000 and test accuracy 0.8803
ENSEMBLE ACCURACY MODE
1.0
ENSEMBLE ACCURACY PROB MEAN ON LOGS
1.0
ENSEMBLE ACCURACY PROB MEAN ON PROBS
1.0

ENSEMBLE ACCURACY MODE
0.9048637092463923
ENSEMBLE ACCURACY PROB MEAN ON LOGS
0.9075360769641903
ENSEMBLE ACCURACY PROB MEAN ON PROBS
0.9048637092463923

array([[-0.85567284, -0.55338029],
       [-0.87034223, -0.54267442],
       [-0.60520196, -0.78957908],
       ...,
       [-1.29842089, -0.31877721],
       [-1.00223294, -0.4573779 ],
       [-0.17982265, -1.80434856]])